# django 불러오기

In [1]:
import os, sys
PWD = os.getenv('PWD')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
init_django('mydjango_project')

# model import

In [48]:
from board.models import Board, Comment

ImportError: cannot import name 'Comment' from 'board.models' (/Users/betweak/betweak/lectures/lecture_pratice/django_practice_list/django_lecture_base_board/mydjango_project/board/models.py)

# Insert or Update

In [24]:
board = Board(
    title="게시글1",
    content="내용1",
)

In [25]:
board.save()

In [26]:
board

<Board: Board object (5)>

-> 내용 확인

In [27]:
board = Board.objects.create(
    title="게시글2",
    content="내용2"
)

In [29]:
board

<Board: Board object (7)>

In [28]:
board = Board.objects.create(
    title="게시글3",
    content="내용3"
)

In [30]:
board

<Board: Board object (7)>

# 조회하기

## raw query로 조회하기

In [32]:
result = Board.objects.raw('SELECT * FROM board_board')
result

<RawQuerySet: SELECT * FROM board_board>

## orm으로 조회하기

### 전체 조회하기

In [12]:
board_list = Board.objects.all()
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>]>

### filter 걸어서 조회하기(where)

In [16]:
board_list = Board.objects.filter(title="게시글")
board_list

<QuerySet [<Board: Board object (4)>]>

In [21]:
board_list = Board.objects.order_by('-created_at') # order by desc
board_list

<QuerySet [<Board: Board object (4)>, <Board: Board object (3)>, <Board: Board object (2)>, <Board: Board object (1)>]>

In [22]:
board_list = Board.objects.order_by('-created_at')[2:4] # order by desc limit(2) offset(2)
board_list

<QuerySet [<Board: Board object (2)>, <Board: Board object (1)>]>

###  filter not 표시

In [33]:
board_list = Board.objects.exclude(title="게시글")
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>]>

In [ ]:
from django.db.models import Q

In [40]:
board_list = Board.objects.filter(~Q(title="게시글"))
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>]>

# 관계 추가하기

1번 댓글에 댓글을 3가지 이상 입력하세요

In [49]:
comment = Comment(
    board_id = 1,
    content = "댓글 내용"
)

NameError: name 'Comment' is not defined

## 

## 조회하기 고급

### 다양한 조건

#### or

In [41]:
board_list = Board.objects.filter(Q(title="게시글1") | Q(title="게시글2"))
board_list

<QuerySet [<Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (5)>, <Board: Board object (6)>]>

##### and 와 연산자

- gte: greater than equal
- lte: less than equal
- lt: less than
- gt: greater than

In [44]:
board_list = Board.objects.filter(Q(title__endswith="글1") & Q(id__gte=3))
board_list

<QuerySet [<Board: Board object (5)>]>

In [46]:
board_list = Board.objects.filter(Q(title__startswith="게시") & Q(created_at__year=2022))
board_list

<QuerySet [<Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>]>

In [126]:


q_4 = Question.objects.filter(~Q(question_text__startswith='django')).all()
q_4

<QuerySet [<Question: Question(1): 뭐가 바뀌었니?>, <Question: Question(3): 마지막 질문이 뭔가요?>, <Question: Question(4): 뭐가 바뀌었니?>, <Question: Question(6): 마지막 질문이 뭔가요?>, <Question: Question(7): 뭐가 바뀌었니?>, <Question: Question(9): 마지막 질문이 뭔가요?>, <Question: Question(10): 뭐가 바뀌었니?>, <Question: Question(12): 마지막 질문이 뭔가요?>]>

# 관계 맺기

In [127]:
q = Question.objects.get(id=1)
q

<Question: Question(1): 뭐가 바뀌었니?>

In [120]:
result = Question.objects.raw('SELECT * FROM polls_question')
result[:2]

[<Question: Question(1): 뭐가 바뀌었니?>, <Question: Question(2): django 재밌어요?>]

In [128]:
c = Choice(
    question=q,
    choice_text="답안1번"
)
c.save()

In [121]:
from django.db.models import Count, Sum
from django.db.models import Q, F

q = Question.objects.annotate(
    max_count=Count('question_text'), changed_name=F('question_text'), sum_id=Sum('id')
).all()
q

<QuerySet [<Question: Question(1): 뭐가 바뀌었니?>, <Question: Question(2): django 재밌어요?>, <Question: Question(3): 마지막 질문이 뭔가요?>, <Question: Question(4): 뭐가 바뀌었니?>, <Question: Question(5): django 재밌어요?>, <Question: Question(6): 마지막 질문이 뭔가요?>, <Question: Question(7): 뭐가 바뀌었니?>, <Question: Question(8): django 재밌어요?>, <Question: Question(9): 마지막 질문이 뭔가요?>, <Question: Question(10): 뭐가 바뀌었니?>, <Question: Question(11): django 재밌어요?>, <Question: Question(12): 마지막 질문이 뭔가요?>]>

In [129]:
c = Choice(
    question=q,
    choice_text="답안2번"
)
c.save()

In [130]:
q2 = Question.objects.get(id=2)

c = Choice(
    question=q2,
    choice_text="답안2-1번"
)
c.save()

In [131]:
Choice.objects.all()

<QuerySet [<Choice: Choice(2): 답안을 바꿔버러요>, <Choice: Choice(3): 답안2-1번>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(6): 답안2-1번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(9): 답안2-1번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.dddsss>, <Choice: Choice(11): 답안1번>, <Choice: Choice(12): 답안2번>, <Choice: Choice(13): 답안2-1번>]>

In [132]:
Choice.objects.filter(question=q)

<QuerySet [<Choice: Choice(2): 답안을 바꿔버러요>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.dddsss>, <Choice: Choice(11): 답안1번>, <Choice: Choice(12): 답안2번>]>

In [133]:
Choice.objects.filter(question_id=q.id)

<QuerySet [<Choice: Choice(2): 답안을 바꿔버러요>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.dddsss>, <Choice: Choice(11): 답안1번>, <Choice: Choice(12): 답안2번>]>

In [134]:
q = Question.objects.get(id=1)
q

<Question: Question(1): 뭐가 바뀌었니?>

In [135]:
# 연관지어 가져오기
q.choice_set.all()

<QuerySet [<Choice: Choice(2): 답안을 바꿔버러요>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.dddsss>, <Choice: Choice(11): 답안1번>, <Choice: Choice(12): 답안2번>]>

In [29]:
q.choice_set.create(choice_text='이렇게 넣을 수도 있어요.', votes=0)

<Choice: Choice(10): 이렇게 넣을 수도 있어요.>

In [30]:
q.choice_set.all()

<QuerySet [<Choice: Choice(1): 답안1번>, <Choice: Choice(2): 답안2번>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.>]>

In [31]:
q.choice_set.count()

7

In [38]:
c1 = q.choice_set.filter(id=1)
c1

<QuerySet [<Choice: Choice(1): 답안1번>]>

In [39]:
c1.delete()

(1, {'polls.Choice': 1})

In [42]:
q.choice_set.all(), q.choice_set.count()

(<QuerySet [<Choice: Choice(2): 답안2번>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.>]>,
 6)

In [57]:
c2 = q.choice_set.get(id=2)
c2

<Choice: Choice(2): 답안2번>

In [66]:
c2.choice_text = "답안을 바꿔버러요"
c2.save()

In [67]:
q.choice_set.all()

<QuerySet [<Choice: Choice(2): 답안을 바꿔버러요>, <Choice: Choice(4): 답안1번>, <Choice: Choice(5): 답안2번>, <Choice: Choice(7): 답안1번>, <Choice: Choice(8): 답안2번>, <Choice: Choice(10): 이렇게 넣을 수도 있어요.>]>

<QuerySet [<Question: Question(9): 마지막 질문이 뭔가요?>, <Question: Question(8): django 재밌어요?>, <Question: Question(7): 뭐가 바뀌었니?>]>

In [25]:
# db 확인하자

In [27]:
q.published_seconds()

'1193 초 전'